In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import numpy as np
from itertools import combinations, groupby
from collections import Counter

In [5]:
orders = pd.read_csv('../input/all_orders.csv', index_col = 0)
cluster_df = pd.read_csv('../input/cluster_df')
products = pd.read_csv('../input/products.csv')
aisles = pd.read_csv('../input/aisles.csv')
departments = pd.read_csv('../input/departments.csv')

c:\users\akshaysharma\hvacproakshay\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
orders.head()

,order_id,product_id,add_to_cart_order,reordered,aisle_id,department_id,aisle,department,user_id,cluster
0,2,33120,1,1,86,16,eggs,dairy eggs,202279,1
1,2,28985,2,1,83,4,fresh vegetables,produce,202279,1
2,2,17794,6,1,83,4,fresh vegetables,produce,202279,1
3,2,43668,9,0,123,4,packaged vegetables fruits,produce,202279,1
4,2,9327,3,0,104,13,spices seasonings,pantry,202279,1


In [7]:
orders = orders.sort_values(['order_id','product_id'])
orders

,order_id,product_id,add_to_cart_order,reordered,aisle_id,department_id,aisle,department,user_id,cluster
30264547,1,10246,3,0,83,4,fresh vegetables,produce,112108,1
30264545,1,11109,2,1,108,16,other creams cheeses,dairy eggs,112108,1
30264548,1,13176,6,0,24,4,fresh fruits,produce,112108,1
30264544,1,22035,8,1,21,16,packaged cheese,dairy eggs,112108,1
30264550,1,43633,5,1,95,15,canned meat seafood,canned goods,112108,1
...,...,...,...,...,...,...,...,...,...,...
23108244,3421083,21162,3,0,92,18,baby food formula,babies,25247,1
23108238,3421083,24852,9,1,24,4,fresh fruits,produce,25247,1
23108247,3421083,35211,5,0,92,18,baby food formula,babies,25247,1
23108242,3421083,39678,6,1,74,17,dish detergents,household,25247,1


In [8]:
# Returns frequency counts for items and item pairs
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")

    
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))


# Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    order_item = order_item.reset_index().to_numpy()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair
            

# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))

In [9]:
def association_rules(order_item, min_support):

    print("Starting order_item: {:22d}".format(len(order_item)))


    # Calculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) 


    # Filter from order_item items below min support 
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Recalculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) 


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders)

    print("Item pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    
    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

In [10]:
# separate order data by cluster and transform into expected format of group association function

cluster_orders = {}
for x in range(0,6):
    i = orders[orders['cluster'] == x]
    i = i.set_index('order_id')['product_id'].rename('item_id')
    cluster_orders[x] = i

In [12]:
# run each cluster series through association rule function and obtain association rules at item level

group_association_rules_dic = {}
for x in range(0,6):
    group_association_rules_dic[x] = association_rules(cluster_orders[x],.0001)

Starting order_item:                1740670
Items with support >= 0.0001:            6216
Remaining order_item:               1631820
Remaining orders with 2+ items:      225372
Remaining order_item:               1609574
Item pairs:                         1643666
Item pairs with support >= 0.0001:      45408

Starting order_item:               15524044
Items with support >= 0.0001:           10316
Remaining order_item:              14600167
Remaining orders with 2+ items:     1276870
Remaining order_item:              14560949
Item pairs:                        14030084
Item pairs with support >= 0.0001:      95546

Starting order_item:               10153348
Items with support >= 0.0001:           13288
Remaining order_item:               9206509
Remaining orders with 2+ items:      992409
Remaining order_item:               9130882
Item pairs:                        15999908
Item pairs with support >= 0.0001:      28714

Starting order_item:                5642242
Items with suppor

In [13]:
# format item association rule dataframes and merge product names

for x in group_association_rules_dic:
    group_association_rules_dic[x] = group_association_rules_dic[x].merge(products[['product_id','product_name']]
               .rename(columns = {'product_id':'item_A','product_name':'product_name_A'}), on = 'item_A')
    group_association_rules_dic[x] = group_association_rules_dic[x].merge(products[['product_id','product_name']]
               .rename(columns = {'product_id':'item_B','product_name':'product_name_B'}), on = 'item_B')
    group_association_rules_dic[x] = group_association_rules_dic[x][['item_A','item_B','product_name_A',
                                                                    'product_name_B','freqAB','supportAB','freqA',
                                                                    'supportA','freqB','supportB','confidenceAtoB',
                                                                    'confidenceBtoA','lift']]
    

In [14]:
# example of association rule dataframe

x = group_association_rules_dic[5]
x.sort_values('lift', ascending = False)

,item_A,item_B,product_name_A,product_name_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,23352,44911,Large Deep Dish Containers + Lids,Entrée Containers & Lids,4,0.000108,4,0.000108,5,0.000135,1.000000,0.800000,7434.000000
1,17950,20259,Vegetable & Pepper Jack Cheese Flatbread Break...,"Special K Sausage, Egg & Cheese Flatbread Brea...",5,0.000135,5,0.000135,5,0.000135,1.000000,1.000000,7434.000000
4465,16241,33002,Peanut Butter Chocolate Chip Meal Bar,Wholeberry Blast Meal Bar,5,0.000135,6,0.000161,5,0.000135,0.833333,1.000000,6195.000000
5109,9620,9909,Hand Made Potato Chips,Big Whole Grain Wheat Thins,5,0.000135,5,0.000135,6,0.000161,1.000000,0.833333,6195.000000
5112,9620,39394,Hand Made Potato Chips,Rich Frosted Donuts,5,0.000135,5,0.000135,6,0.000161,1.000000,0.833333,6195.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6534,19660,36316,Spring Water,Lemon Sparkling Water,4,0.000108,3608,0.097068,1393,0.037476,0.001109,0.002872,0.029583
2694,8859,44632,Natural Spring Water,Sparkling Water Grapefruit,6,0.000161,1313,0.035324,5941,0.159833,0.004570,0.001010,0.028590
4390,19660,35221,Spring Water,Lime Sparkling Water,11,0.000296,3608,0.097068,4168,0.112133,0.003049,0.002639,0.027189
20311,19660,43154,Spring Water,Sparkling Mineral Water,4,0.000108,3608,0.097068,1543,0.041512,0.001109,0.002592,0.026707


In [15]:
# separate order data by cluster and transform into expected format of group association function at aisle level

cluster_aisle_orders = {}
for x in range(0,6):
    i = orders[orders['cluster'] == x]
    i = i.groupby(['order_id','aisle_id'])['product_id'].count().reset_index().set_index('order_id')['aisle_id'].rename('item_id')
    cluster_aisle_orders[x] = i

In [16]:
orders.groupby(['order_id','aisle_id'])['product_id'].count().reset_index().set_index('order_id')['aisle_id'].rename('item_id')

order_id
1           21
1           24
1           83
1           95
1          108
          ... 
3421083     52
3421083     74
3421083     78
3421083     92
3421083    117
Name: item_id, Length: 24329222, dtype: int64

In [17]:
orders[orders['cluster'] == 5]['order_id'].nunique()

48794

In [18]:
# run association function at aisle level

group_aisle_association_rules_dic = {}
for x in range(0,6):
    group_aisle_association_rules_dic[x] = association_rules(cluster_aisle_orders[x],.0001)

Starting order_item:                1184688
Items with support >= 0.0001:             134
Remaining order_item:               1184688
Remaining orders with 2+ items:      219857
Remaining order_item:               1155194
Item pairs:                            8554
Item pairs with support >= 0.0001:       5396

Starting order_item:               11211683
Items with support >= 0.0001:             134
Remaining order_item:              11211683
Remaining orders with 2+ items:     1281160
Remaining order_item:              11171978
Item pairs:                            8906
Item pairs with support >= 0.0001:       7573

Starting order_item:                7680091
Items with support >= 0.0001:             134
Remaining order_item:               7680091
Remaining orders with 2+ items:      991382
Remaining order_item:               7595005
Item pairs:                            8908
Item pairs with support >= 0.0001:       8202

Starting order_item:                3694150
Items with suppor

In [19]:
# reformat cluster aisle assocation dataframes

for x in group_aisle_association_rules_dic:
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].merge(aisles[['aisle_id','aisle']]
               .rename(columns = {'aisle_id':'item_A','aisle':'aisle_name_A'}), on = 'item_A')
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].merge(aisles[['aisle_id','aisle']]
               .rename(columns = {'aisle_id':'item_B','aisle':'aisle_name_B'}), on = 'item_B')
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x][['item_A','item_B','aisle_name_A',
                                                                    'aisle_name_B','freqAB','supportAB','freqA',
                                                                    'supportA','freqB','supportB','confidenceAtoB',
                                                                    'confidenceBtoA','lift']]
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].rename(columns = {'item_A':'aisle_A','item_B':'aisle_B'})

In [20]:
x = group_aisle_association_rules_dic[5].sort_values('lift',ascending = False)
x.sort_values('lift',ascending = False)

,aisle_A,aisle_B,aisle_name_A,aisle_name_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,28,62,red wines,white wines,50,0.001467,115,0.003374,168,0.004929,0.434783,0.297619,88.211698
15,28,134,red wines,specialty wines champagnes,12,0.000352,115,0.003374,77,0.002259,0.104348,0.155844,46.190853
1401,40,119,dog food care,frozen dessert,5,0.000147,70,0.002054,54,0.001584,0.071429,0.092593,45.085979
16,124,134,spirits,specialty wines champagnes,11,0.000323,117,0.003433,77,0.002259,0.094017,0.142857,41.617827
1840,65,109,protein meal replacements,skin care,5,0.000147,125,0.003667,33,0.000968,0.040000,0.151515,41.315152
...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,46,83,mint gum,fresh vegetables,4,0.000117,439,0.012880,1618,0.047470,0.009112,0.002472,0.191947
3909,83,125,fresh vegetables,trail mix snack mix,6,0.000176,1618,0.047470,673,0.019745,0.003708,0.008915,0.187811
1464,32,64,packaged produce,energy sports drinks,7,0.000205,1438,0.042189,903,0.026493,0.004868,0.007752,0.183745
1488,32,48,packaged produce,breakfast bars pastries,5,0.000147,1438,0.042189,658,0.019305,0.003477,0.007599,0.180114


In [21]:
cluster_df['cluster'].value_counts() / cluster_df['cluster'].value_counts().sum()

2    0.356837
1    0.352123
3    0.159280
0    0.074749
4    0.038543
5    0.018467
Name: cluster, dtype: float64

In [24]:
for x in group_association_rules_dic:
    group_association_rules_dic[x].to_csv('../input/associationRule/group'+str(x)+'_item_rules.csv')

In [25]:
for x in group_aisle_association_rules_dic:
    group_aisle_association_rules_dic[x].to_csv('../input/associationRule/group'+str(x)+'_aisle_rules.csv')